In [0]:
ADLS_SILVER = "abfss://citibike@databricksjm.dfs.core.windows.net/silver"

df = (
    spark.read
    .format("delta")
    .load(ADLS_SILVER)
    .sample(False, fraction=0.005)
)
display(df)

In [0]:
from pyspark.sql.functions import col, min, max, count

df.count()

In [0]:
display(
    df.select(
        min(col("year")),
        max(col("year"))
    )
)

In [0]:
display(
    df.select(
        "year"
    )
    .distinct()
    .orderBy("year")
)

In [0]:
display(
    df
    .orderBy(
        col("duration_sec"),
        ascending=False
    )
)

In [0]:
def duration_filter(df, duration_min=59, duration_max=89736):
    df = (
        df
        .filter(
            (col("duration_sec") > duration_min) &
            (col("duration_sec") < duration_max)
        )
    )
    return df

In [0]:
display(
    df
    .select(
        "start_station_name"
    )
    .distinct()
)

In [0]:
from pyspark.sql.functions import col

def top_stations(df, n:int = 10):
    

    df_start_stations = (
        df
        .select(
            col("start_station_name").alias("station_name")
        )
    )
    df_end_stations = (
        df
        .select(
            col("end_station_name").alias("station_name")
        )
    )
    df_stations = (
        df_start_stations
        .union(df_end_stations)
        .filter(col("station_name").isNotNull())
    )

    df_top_stations = (
        df_stations
        .groupBy("station_name")
        .agg(
            count("*").alias("total_uses")
        )
        .orderBy(
            col("total_uses")
            .desc()
        )
        .limit(n)
    )
    return df_top_stations
    

In [0]:
df_start_stations = (
    df
    .select(
        col("start_station_name").alias("station_name")
    )
)
df_end_stations = (
    df
    .select(
        col("end_station_name").alias("station_name")
    )
)
df_stations = (
    df_start_stations
    .union(df_end_stations)
    .filter(col("station_name").isNotNull())
)

df = (
    df_stations
    .groupBy("station_name")
    .agg(
        count("*").alias("total_uses")
    )
    .orderBy(
        col("total_uses")
        .desc()
    )
    .limit(10)
)
display(df)